In [ ]:
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook as tqdm

from sklearn.model_selection import StratifiedKFold

In [ ]:
sns.set_context('talk')

# Load data

In [ ]:
df = pd.read_csv('data/train.csv')

In [ ]:
cat_vars = ['customer', 'category', 'suspicious', 'nationality', 'is_pep']
for var_ in cat_vars:
    df[var_] = df[var_].astype('category')

In [ ]:
df['suspicious'].astype(int).sum()

In [ ]:
df.head()

# Basic statistics

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
sns.boxplot(df['age'])

# Model

In [ ]:
test_features = pd.read_csv('data/test.csv')

In [ ]:
test_features.shape

## SVM

In [ ]:
from sklearn import svm

In [ ]:
sub = df.sample(n=100_000)
sub['suspicious'].astype(int).sum()

### Train

In [ ]:
X = sub[['nationality', 'atm_withdrawal']]
y = sub['suspicious']

In [ ]:
X.shape

In [ ]:
skf = StratifiedKFold(n_splits=2)

In [ ]:
%%time
for train_index, test_index in tqdm(skf.split(X, y), total=skf.get_n_splits(X, y)):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    clf = svm.SVC(gamma='scale', class_weight='balanced')
    clf.fit(X_train, y_train)
    
    print(clf.score(X_test, y_test))

In [ ]:
%%time
clf = svm.SVC(kernel='rbf', gamma='scale', class_weight='balanced')
clf.fit(X, y)

### Test

In [ ]:
sub_test = test_features[['nationality', 'atm_withdrawal']]
sub_test.shape

In [ ]:
%%time
predictions = clf.predict(sub_test)

In [ ]:
predictions.sum()

### Save results

In [ ]:
# TODO